In [1]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveUS.p", "rb" ) )


In [2]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,2,56.0
1,0,10,5.0
2,0,15,3.0
3,0,26,14.0
4,0,31,2.0


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [4]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>24)


In [5]:
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)

In [6]:
ratings =ratingsPD2

In [7]:
len(ratings)

176347

In [8]:
ratings.groupby('user').count()

,item,rating
user,,
0,511,511
1,656,656
2,590,590
3,53,53
4,240,240
5,1185,1185
6,320,320
7,317,317
8,127,127


In [9]:
ratings['item'].nunique()

13593

In [10]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0.4289146174730043
0.46759926800303886
0.4388448229904807
0.424865884891785
0.43967037253858043
Average RMSE: 0.43997899317937783


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))